In [20]:
# Step 0: Setup
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv("../.env")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("LLM ready")

%load_ext autoreload
%autoreload 2

LLM ready
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Step 1: Load the raw data and see what's messy

In [21]:
df_raw = pd.read_csv("../data/sample_data.csv")
print(f"Shape: {df_raw.shape}")
print(f"\nMissing values:\n{df_raw.isna().sum()}")
print(f"\nDuplicate rows: {df_raw.duplicated().sum()}")
df_raw

Shape: (96, 7)

Missing values:
name            0
age             1
salary          4
department      0
experience      1
city            8
employee_id    92
dtype: int64

Duplicate rows: 1


,name,age,salary,department,experience,city,employee_id
0,John Doe,28.0,50000.0,Sales,3.0,New York,E001
1,Jane Smith,34.0,65000.0,Marketing,7.0,Boston,NaN
2,Bob Johnson,45.0,NaN,IT,15.0,NaN,NaN
3,Alice Brown,29.0,48000.0,Sales,2.0,Chicago,NaN
4,Charlie Davis,NaN,NaN,HR,NaN,NaN,NaN
...,...,...,...,...,...,...,...
91,Lily Ward,35.0,62000.0,HR,9.0,Austin,NaN
92,Mark Cole,30.0,56000.0,Sales,6.0,Portland,NaN
93,Nora Moon,39.0,69000.0,Marketing,13.0,Phoenix,NaN
94,Omar Lane,26.0,47000.0,IT,2.0,Dallas,NaN


## Step 2: Build the summary the LLM sees
This is what `get_dataframe_summary()` produces — the LLM's only view of your data.

In [22]:
from utils import get_dataframe_summary

summary = get_dataframe_summary(df_raw, indent=4)
print(summary)

    Column Data Types:
      name: object
      age: float64
      salary: float64
      department: object
      experience: float64
      city: object
      employee_id: object

    Missing Value Percentage:
      employee_id: 95.83%
      city: 8.33%
      salary: 4.17%
      age: 1.04%
      experience: 1.04%
      name: 0.00%
      department: 0.00%

    Outlier Stats (numerical cols):
      age: Lower Bound: 3.05, Upper Bound: 64.25
      salary: Lower Bound: 7375.00, Upper Bound: 109175.00
      experience: Lower Bound: -18.70, Upper Bound: 35.70

    String Column Samples:
      name: ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Brown', 'Charlie Davis', 'Emma Wilson', 'Frank Miller', 'Grace Lee', 'Henry Taylor', 'Ivy Chen']
      department: ['Sales', 'Marketing', 'IT', 'HR']
      city: ['New York', 'Boston', 'Chicago', 'Seattle', 'Austin', 'Denver', 'Portland', 'Miami', 'Phoenix', 'Atlanta']
      employee_id: ['E001', 'E007', 'E019']


## Step 3: Build the prompt and send it to the LLM
This is the actual prompt the model receives. It asks for a Python function.

In [23]:
function_name="data_cleaner"

In [24]:
from utils import DATA_CLEANING_PROMPT_TEMPLATE

prompt = PromptTemplate(
    template=DATA_CLEANING_PROMPT_TEMPLATE,
    input_variables=["user_instructions", "all_datasets_summary", "function_name"]
)

In [29]:
filled_prompt = prompt.format(
    user_instructions="Follow the basic cleaning steps.",
    all_datasets_summary=summary,
    function_name=function_name
)
print(filled_prompt)

You are a Data Cleaning Agent. Create a data_cleaner() function to clean the data.

Basic Cleaning Steps to implement:
1. Remove columns with more than 40% missing values
2. Impute missing values (mean for numeric, mode for categorical)
3. Remove duplicate rows
4. Remove outliers (numerical cols) outside of p05 and p95
5. Normalize string columns (strip whitespace, normalize casing)

User Instructions:
Follow the basic cleaning steps.

Dataset Summary:
    Column Data Types:
      name: object
      age: float64
      salary: float64
      department: object
      experience: float64
      city: object
      employee_id: object

    Missing Value Percentage:
      employee_id: 95.83%
      city: 8.33%
      salary: 4.17%
      age: 1.04%
      experience: 1.04%
      name: 0.00%
      department: 0.00%

    Outlier Stats (numerical cols):
      age: Lower Bound: 3.05, Upper Bound: 64.25
      salary: Lower Bound: 7375.00, Upper Bound: 109175.00
      experience: Lower Bound: -18.70, Up

## Step 4: Call the LLM and parse the Python code out
The LCEL chain `prompt | llm | parser` does this in one call.

In [ ]:
prompt

In [ ]:
llm

In [ ]:
from utils import PythonOutputParser

chain = prompt | llm | PythonOutputParser()


In [ ]:
type(chain)

In [ ]:

generated_code = chain.invoke({
    "user_instructions": "Follow the basic cleaning steps.",
    "all_datasets_summary": summary,
})

print(generated_code)

## Step 5: Execute the generated code with `exec()`
This is what `execute_agent_code()` does under the hood — runs the code string, extracts the function, and calls it.

In [ ]:
# exec() runs the code string and defines the function in local_vars
local_vars = {}
exec(generated_code, {}, local_vars)

# Pull out the function by name
data_cleaner = local_vars["data_cleaner"]
print(f"Got function: {data_cleaner}")

# Call it on our raw data
df_cleaned = data_cleaner(df_raw)

print(f"\nBefore: {df_raw.shape} -> After: {df_cleaned.shape}")
print(f"Missing values after cleaning:\n{df_cleaned.isna().sum()}")
print(f"Duplicate rows: {df_cleaned.duplicated().sum()}")
df_cleaned

## Step 6: Now run it all at once through the agent
This is equivalent to what `app.py` does — the agent handles steps 2-5 internally.

In [ ]:
import sys
sys.path.insert(0, "..")

from data_cleaning_agent import LightweightDataCleaningAgent

agent = LightweightDataCleaningAgent(model=llm, log=True, log_path="../logs/")
agent.invoke_agent(data_raw=df_raw)

df_result = agent.get_data_cleaned()
print(f"Before: {df_raw.shape} -> After: {df_result.shape}")
df_result